In [52]:
import json

with open('bench.bril', 'r') as file:
    bril_file = json.load(file)

In [53]:
def find_op(bril_file, op):
    add_instrs = []
    
    functions = bril_file.get('functions', [])

    for function in functions:
        instrs = function.get('instrs', [])
        num_instrs = len(instrs)
        
        print(f"Function '{function['name']}' has {num_instrs} instructions.")
        
        for instr in instrs:
            if instr.get('op') == op:
                add_instrs.append(instr)

    return add_instrs

find_op(bril_file, "br")

Function 'main' has 6 instructions.
Function 'benchmark' has 50 instructions.


[{'op': 'br', 'args': ['v7'], 'labels': ['for.body.3', 'for.end.3']},
 {'op': 'br', 'args': ['v20'], 'labels': ['then.17', 'else.17']}]

In [54]:
def count_instruction_types(bril_file):
    instruction_count = {}
    
    functions = bril_file.get('functions', [])

    for function in functions:
        instrs = function.get('instrs', [])
        
        for instr in instrs:
            op_type = instr.get('op')
            if op_type:

                instruction_count[op_type] = instruction_count.get(op_type, 0) + 1

    return instruction_count

count_instruction_types(bril_file)

{'const': 10,
 'call': 1,
 'id': 25,
 'print': 1,
 'lt': 1,
 'br': 2,
 'add': 3,
 'mul': 1,
 'sub': 2,
 'eq': 1,
 'jmp': 2,
 'ret': 1}

In [57]:
def insert_before_op_code(bril_file, target_op_code):
    functions = bril_file.get('functions', [])

    for function in functions:
        instrs = function.get('instrs', [])

        modified_instrs = []

        for instr in instrs:
            if instr.get('op') == target_op_code:
                modified_instrs.append({"op": "id", 
                                        "dest": "v30", 
                                        "type": "int", 
                                        "args": ["i"]})
                                        
                modified_instrs.append({"op": "print", 
                                         "args": ["v30"]})

            modified_instrs.append(instr)

        function['instrs'] = modified_instrs

    return bril_file

with open('bench.bril', 'r') as file:
    bril_file = json.load(file)

print_after_jmp = insert_before_op_code(bril_file, "jmp")
print(print_after_jmp)

with open("bench-print-before-jmp.bril", 'w') as file:
    json.dump(print_after_jmp, file)


{'functions': [{'name': 'main', 'instrs': [{'op': 'const', 'value': 10, 'dest': 'v31', 'type': 'int'}, {'op': 'call', 'dest': 'result', 'type': 'int', 'args': ['v31'], 'funcs': ['benchmark']}, {'op': 'id', 'dest': 'result', 'type': 'int', 'args': ['result']}, {'op': 'id', 'dest': 'v32', 'type': 'int', 'args': ['result']}, {'op': 'print', 'args': ['v32']}, {'op': 'const', 'value': 0, 'dest': 'v33', 'type': 'int'}], 'args': []}, {'name': 'benchmark', 'instrs': [{'op': 'const', 'value': 0, 'dest': 'v0', 'type': 'int'}, {'op': 'id', 'dest': 'sum', 'type': 'int', 'args': ['v0']}, {'op': 'const', 'value': 1, 'dest': 'v1', 'type': 'int'}, {'op': 'id', 'dest': 'product', 'type': 'int', 'args': ['v1']}, {'op': 'const', 'value': 1, 'dest': 'v2', 'type': 'int'}, {'op': 'id', 'dest': 'flag', 'type': 'int', 'args': ['v2']}, {'op': 'const', 'value': 0, 'dest': 'v4', 'type': 'int'}, {'op': 'id', 'dest': 'i', 'type': 'int', 'args': ['v4']}, {'label': 'for.cond.3'}, {'op': 'id', 'dest': 'v5', 'type': '